In [44]:
import os
import pandas as pd
import numpy as np
import time
import yfinance as yf

In [3]:
path = '/Users/Marcos/Documents/myst_lab1_MOTM/files/NAFTRAC_holdings'
abspath = os.path.abspath(path)
archivos = [f[:-4] for f in os.listdir(abspath) if os.path.isfile(os.path.join(abspath, f))]

In [14]:
data_archivos = {}
for i in archivos:
    # i = achivos[0]
    data = pd.read_csv('/Users/Marcos/Documents/myst_lab1_MOTM/files/NAFTRAC_holdings/' + i + '.csv', skiprows=2, header=None) # leer archivos despues de los primeros 2 renglones
    data.columns = list(data.iloc[0, :]) # renombrar columnas
    data = data.loc[:, pd.notnull(data.columns)]
    data = data.iloc[1:-1].reset_index(drop=True, inplace=False)
    data['Precio'] = [i.replace(',','') for i in data['Precio']] # quitar comas
    data['Ticker'] = [i.replace('*','') for i in data['Ticker']] # quitar asteriscos
    convert_dict = {'Ticker': str, 'Nombre': str, 'Peso (%)': float, 'Precio': float}
    data = data.astype(convert_dict)
    data['Peso (%)'] = data['Peso (%)']/100 # convertir a decimal la columna de peso
    data_archivos[i] = data # guardar en diccionario

In [96]:
t_fechas = [i.strftime('%d-%m-%Y') for i in sorted([pd.to_datetime(i[8:]).date() for i in archivos])] # sirve como etiqueta en dataframe y para yfinance
i_fechas = [j.strftime('%Y-%m-%d') for j in sorted([pd.to_datetime(i[8:]).date() for i in archivos])] #lista con fechas ordenadas para usarse como indexadores de archivos


In [308]:
# -- descargar y acomodar los datos
tickers = []
for i in archivos:
    # i = archivos[1]
    l_tickers = list(data_archivos[i]['Ticker'])
    [tickers.append(i + '.MX') for i in l_tickers]
global_tickers = np.unique(tickers).tolist() #lista global (todos los datos que vamos a necesitar para descargar)

In [309]:
global_tickers = [i.replace('GFREGIOO.MX', 'RA.MX') for i in global_tickers]
global_tickers = [i.replace('MEXCHEM.MX', 'ORBIA.MX') for i in global_tickers]
global_tickers = [i.replace('LIVEPOLC.1.MX', 'LIVEPOLC-1.MX') for i in global_tickers]

[global_tickers.remove(i) for i in ['MXN.MX', 'USD.MX', 'KOFL.MX', 'KOFUBL.MX', 'BSMXB.MX']]

[None, None, None, None, None]

In [439]:
global_tickers

['AC.MX',
 'ALFAA.MX',
 'ALPEKA.MX',
 'ALSEA.MX',
 'AMXL.MX',
 'ASURB.MX',
 'BBAJIOO.MX',
 'BIMBOA.MX',
 'BOLSAA.MX',
 'CEMEXCPO.MX',
 'CUERVO.MX',
 'ELEKTRA.MX',
 'FEMSAUBD.MX',
 'GAPB.MX',
 'GCARSOA1.MX',
 'GCC.MX',
 'GENTERA.MX',
 'GFINBURO.MX',
 'GFNORTEO.MX',
 'RA.MX',
 'GMEXICOB.MX',
 'GMXT.MX',
 'GRUMAB.MX',
 'IENOVA.MX',
 'KIMBERA.MX',
 'LABB.MX',
 'LALAB.MX',
 'LIVEPOLC-1.MX',
 'MEGACPO.MX',
 'ORBIA.MX',
 'NEMAKA.MX',
 'OMAB.MX',
 'ORBIA.MX',
 'PE&OLES.MX',
 'PINFRA.MX',
 'Q.MX',
 'RA.MX',
 'TLEVISACPO.MX',
 'VESTA.MX',
 'VOLARA.MX',
 'WALMEX.MX']

In [311]:
inicio = time.time()
# descarga de yahoofinance
data = yf.download(global_tickers, start="2017-08-21", end="2020-08-24", actions=False,
                   group_by="close", interval='1d', auto_adjust=False, prepost=False, threads=True)

print('se tardo', round(time.time()-inicio, 2), 'segundos')

[*********************100%***********************]  39 of 39 completed
se tardo 5.17 segundos


In [312]:
#convertir columna de fechas
data_close = pd.DataFrame({i: data[i]['Close'] for i in global_tickers})

In [313]:
data_close.head()

,AC.MX,ALFAA.MX,ALPEKA.MX,ALSEA.MX,AMXL.MX,ASURB.MX,BBAJIOO.MX,BIMBOA.MX,BOLSAA.MX,CEMEXCPO.MX,...,ORBIA.MX,NEMAKA.MX,OMAB.MX,PE&OLES.MX,PINFRA.MX,Q.MX,TLEVISACPO.MX,VESTA.MX,VOLARA.MX,WALMEX.MX
Date,,,,,,,,,,,,,,,,,,,,,
2017-08-21,135.740005,24.469999,19.730000,68.330002,16.639999,360.589996,36.240002,43.400002,31.559999,16.240000,...,47.990002,15.02,109.269997,455.899994,199.139999,31.469999,90.870003,27.400000,23.010000,44.090000
2017-08-22,133.960007,25.080000,19.570000,68.000000,16.660000,361.200012,36.580002,43.619999,31.700001,16.200001,...,48.169998,15.20,108.269997,451.369995,198.369995,31.990000,90.809998,27.230000,22.790001,43.560001
2017-08-23,133.550003,24.580000,19.549999,67.750000,16.549999,358.209991,36.959999,43.750000,31.980000,16.150000,...,48.250000,14.84,109.290001,455.149994,195.919998,32.290001,91.230003,27.410000,22.690001,43.990002
2017-08-24,131.679993,25.030001,19.620001,66.660004,16.650000,356.390015,36.700001,44.150002,31.650000,16.110001,...,48.020000,14.81,109.820000,461.790009,195.429993,32.000000,91.599998,26.930000,22.650000,44.330002
2017-08-25,130.130005,25.370001,20.360001,65.730003,16.639999,357.250000,36.619999,44.419998,31.750000,16.160000,...,48.619999,14.71,108.419998,466.739990,195.679993,32.009998,92.720001,26.879999,23.059999,43.669998


In [314]:
# tomar solo las fechas de interes
ic_fechas = sorted(list(set(data_close.index.astype(str).tolist()) & set(i_fechas)))

In [315]:
#localizar todos los precios (encontrar todos los precios de cada mes)
precios = data_close.loc[data_close.index.astype(str).isin(ic_fechas)]

In [316]:
precios.head()

,AC.MX,ALFAA.MX,ALPEKA.MX,ALSEA.MX,AMXL.MX,ASURB.MX,BBAJIOO.MX,BIMBOA.MX,BOLSAA.MX,CEMEXCPO.MX,...,ORBIA.MX,NEMAKA.MX,OMAB.MX,PE&OLES.MX,PINFRA.MX,Q.MX,TLEVISACPO.MX,VESTA.MX,VOLARA.MX,WALMEX.MX
Date,,,,,,,,,,,,,,,,,,,,,
2018-01-31,135.050003,23.360001,26.860001,60.880001,17.440001,362.220001,39.330002,45.400002,35.619999,15.48,...,52.750000,15.54,94.989998,431.350006,190.779999,46.500000,77.019997,25.639999,15.930000,46.639999
2018-02-28,130.839996,22.389999,24.420000,64.260002,17.340000,334.029999,41.049999,44.110001,34.939999,12.45,...,52.840000,13.41,92.110001,411.940002,192.380005,49.580002,64.510002,25.900000,16.780001,44.000000
2018-03-28,125.709999,23.290001,25.260000,63.529999,17.250000,306.420013,38.910000,39.779999,34.740002,12.03,...,55.639999,14.94,88.650002,367.260010,180.740005,50.320000,57.830002,26.990000,14.650000,46.259998
2018-04-30,129.259995,24.010000,26.809999,69.559998,17.309999,336.329987,39.889999,43.529999,36.080002,11.70,...,58.419998,14.11,99.040001,392.540009,191.639999,52.209999,67.120003,27.000000,11.690000,51.970001
2018-05-31,119.680000,20.480000,29.750000,65.410004,15.500000,316.660004,37.799999,38.060001,32.509998,11.86,...,56.189999,13.50,95.379997,336.880005,179.300003,51.049999,66.599998,25.070000,10.300000,50.060001


In [317]:
precios = precios.reindex(sorted(precios.columns), axis=1)
precios.head()

,AC.MX,ALFAA.MX,ALPEKA.MX,ALSEA.MX,AMXL.MX,ASURB.MX,BBAJIOO.MX,BIMBOA.MX,BOLSAA.MX,CEMEXCPO.MX,...,OMAB.MX,ORBIA.MX,PE&OLES.MX,PINFRA.MX,Q.MX,RA.MX,TLEVISACPO.MX,VESTA.MX,VOLARA.MX,WALMEX.MX
Date,,,,,,,,,,,,,,,,,,,,,
2018-01-31,135.050003,23.360001,26.860001,60.880001,17.440001,362.220001,39.330002,45.400002,35.619999,15.48,...,94.989998,52.750000,431.350006,190.779999,46.500000,115.919998,77.019997,25.639999,15.930000,46.639999
2018-02-28,130.839996,22.389999,24.420000,64.260002,17.340000,334.029999,41.049999,44.110001,34.939999,12.45,...,92.110001,52.840000,411.940002,192.380005,49.580002,116.150002,64.510002,25.900000,16.780001,44.000000
2018-03-28,125.709999,23.290001,25.260000,63.529999,17.250000,306.420013,38.910000,39.779999,34.740002,12.03,...,88.650002,55.639999,367.260010,180.740005,50.320000,111.919998,57.830002,26.990000,14.650000,46.259998
2018-04-30,129.259995,24.010000,26.809999,69.559998,17.309999,336.329987,39.889999,43.529999,36.080002,11.70,...,99.040001,58.419998,392.540009,191.639999,52.209999,119.480003,67.120003,27.000000,11.690000,51.970001
2018-05-31,119.680000,20.480000,29.750000,65.410004,15.500000,316.660004,37.799999,38.060001,32.509998,11.86,...,95.379997,56.189999,336.880005,179.300003,51.049999,111.129997,66.599998,25.070000,10.300000,50.060001


In [523]:
# posicion inicial

k = 1000000 #capital inicial
c = 0.0125 #comision
comisiones = [] #vector de comisiones historicas

In [524]:
# obtener posicion inicial, los % para KOFL, KOFUBL, BSMXB, USD asignarlos a cash
c_activos = ['KOFL', 'KOFUBL', 'BSMXB', 'MXN', 'USD']

#diccionario para resultado final
inv_pasiva = {'timestamp': ['2018-01-30'], 'capital': [k]}

# revisar que sea el archivo del primer mes
pos_datos = data_archivos['NAFTRAC_310118'].copy().sort_values('Ticker')[['Ticker', 'Nombre', 'Peso (%)']]

In [525]:
pos_datos

,Ticker,Nombre,Peso (%)
13,AC,ARCA CONTINENTAL,0.0175
7,ALFAA,ALFA A,0.0268
33,ALPEKA,ALPEK SA DE CV,0.0032
26,ALSEA,ALSEA DE CV,0.0083
0,AMXL,AMERICA MOVIL L,0.1331
9,ASURB,GRUPO AEROPORTUARIO DEL SURESTE B,0.0244
11,BIMBOA,GRUPO BIMBO A,0.0178
16,BSMXB,BANCO SANTANDER B,0.0166
5,CEMEXCPO,CEMEX CPO,0.0744
27,CUERVO,BECLE SA DE CV SA,0.0062


In [526]:
# extraer la lista de activos a eliminar
i_activos = list(pos_datos[list(pos_datos['Ticker'].isin(c_activos))].index)

In [527]:
# eliminar los activos del dataframe
pos_datos.drop(i_activos, inplace=True)

In [528]:
# agregar .MX para empatar precios
pos_datos['Ticker'] = pos_datos['Ticker'] + '.MX'

In [529]:
#corregir tickers en datos
pos_datos['Ticker'] = pos_datos['Ticker'].replace('LIVEPOLC.1.MX', 'LIVEPOLC-1.MX')
pos_datos['Ticker'] = pos_datos['Ticker'].replace('MEXCHEM.MX', 'ORBIA.MX')
pos_datos['Ticker'] = pos_datos['Ticker'].replace('GFREGIOO.MX', 'RA.MX')

In [530]:
pos_datos

,Ticker,Nombre,Peso (%)
13,AC.MX,ARCA CONTINENTAL,0.0175
7,ALFAA.MX,ALFA A,0.0268
33,ALPEKA.MX,ALPEK SA DE CV,0.0032
26,ALSEA.MX,ALSEA DE CV,0.0083
0,AMXL.MX,AMERICA MOVIL L,0.1331
9,ASURB.MX,GRUPO AEROPORTUARIO DEL SURESTE B,0.0244
11,BIMBOA.MX,GRUPO BIMBO A,0.0178
5,CEMEXCPO.MX,CEMEX CPO,0.0744
27,CUERVO.MX,BECLE SA DE CV SA,0.0062
19,ELEKTRA.MX,GRUPO ELEKTRA,0.0136


In [531]:
#fecha para la que se busca hacer el match de precios
match = 0
precios.index.to_list()[match]

Timestamp('2018-01-31 00:00:00')

In [532]:
#precios necesarios para la posicion metodo 2
precio_inicial = [precios.iloc[match, precios.columns.to_list().index(i)] for i in pos_datos['Ticker']]

In [533]:
pos_datos['Precio'] = precio_inicial

In [534]:
# capital destinado  por accion = proporcion de capital - comisiones por la posicion
capital = pos_datos['Peso (%)']*k - pos_datos['Peso (%)']*k*c

In [535]:
# cantidad de titulos por accion
pos_datos['Titulos'] = capital//pos_datos['Precio']

In [536]:
#valor de la postura por accion
pos_datos['Postura'] = pos_datos['Titulos']*pos_datos['Precio']

In [537]:
#comision pagada
pos_datos['Comision'] = pos_datos['Postura']*c
pos_comision = pos_datos['Comision'].sum()

In [538]:
#efectivo libre en la postura
pos_cash = k - pos_datos['Postura'].sum() - pos_comision
#valor de la posicion
pos_value = pos_datos['Postura'].sum()

In [539]:
pos_datos

,Ticker,Nombre,Peso (%),Precio,Titulos,Postura,Comision
13,AC.MX,ARCA CONTINENTAL,0.0175,135.050003,127.0,17151.350388,214.391880
7,ALFAA.MX,ALFA A,0.0268,23.360001,1132.0,26443.520691,330.544009
33,ALPEKA.MX,ALPEK SA DE CV,0.0032,26.860001,117.0,3142.620071,39.282751
26,ALSEA.MX,ALSEA DE CV,0.0083,60.880001,134.0,8157.920143,101.974002
0,AMXL.MX,AMERICA MOVIL L,0.1331,17.440001,7536.0,131427.844025,1642.848050
9,ASURB.MX,GRUPO AEROPORTUARIO DEL SURESTE B,0.0244,362.220001,66.0,23906.520081,298.831501
11,BIMBOA.MX,GRUPO BIMBO A,0.0178,45.400002,387.0,17569.800591,219.622507
5,CEMEXCPO.MX,CEMEX CPO,0.0744,15.480000,4746.0,73468.077827,918.350973
27,CUERVO.MX,BECLE SA DE CV SA,0.0062,33.860001,180.0,6094.800110,76.185001
19,ELEKTRA.MX,GRUPO ELEKTRA,0.0136,672.390015,19.0,12775.410278,159.692628


In [540]:
# actualizar lista de valores de cada llave en el diccionario
inv_pasiva['timestamp'].append(i_fechas[0])
inv_pasiva['capital'].append(pos_value + pos_cash)

In [541]:
inv_pasiva

{'timestamp': ['2018-01-30', '2018-01-31'],
 'capital': [1000000, 988201.7086287022]}

In [542]:
match = 1
for match in range(len(precios)):
    precios_i = [precios.iloc[match, precios.columns.to_list().index(i)] for i in pos_datos['Ticker']]
    capital_i = sum(pos_datos['Titulos'] * precios_i)
    
    inv_pasiva['timestamp'].append(precios.index.astype(str).to_list()[match])
    inv_pasiva['capital'].append(capital_i + pos_cash)
    


In [548]:
df_pasiva = pd.DataFrame(inv_pasiva)
df_pasiva['rend'] = df_pasiva['capital'].pct_change()
df_pasiva['rend_acum'] = df_pasiva['rend'].cumsum()
df_pasiva = df_pasiva.fillna(0)
df_pasiva = df_pasiva.set_index('timestamp') 

In [549]:
df_pasiva

,capital,rend,rend_acum
timestamp,,,
2018-01-30,1000000.000000,0.000000,0.000000
2018-01-31,988201.708629,-0.011798,-0.011798
2018-01-31,988201.708629,0.000000,-0.011798
2018-02-28,933453.795098,-0.055402,-0.067200
2018-03-28,909385.391446,-0.025784,-0.092984
2018-04-30,951725.438125,0.046559,-0.046425
2018-05-31,881316.754988,-0.073980,-0.120405
2018-06-29,939980.274362,0.066563,-0.053842
2018-07-31,975386.766531,0.037667,-0.016174
